In [ ]:
#@title Ejecutar primero (reiniciará el entorno)
from IPython.utils import io
with io.capture_output() as captured:
    !pip install scikit-learn==0.23.2 --upgrade
exit()

In [ ]:
#@title Predicción
import pickle
import requests
import sklearn
import ipywidgets as wd
import numpy as np
from IPython.display import display, clear_output

def obtener_clasificadores():
    archivo = requests.get('https://drive.google.com/u/0/uc?id=1k6BVa1oTeqT84zqEE8BOqIHkkFVtzLum&export=download').content
    clasificadores = pickle.loads(archivo)
    return clasificadores

def convertir_a_variante(cod_variante, data):
    fecha = data[0][0]
    if cod_variante == 1:
        return [[fecha.day]]
    elif cod_variante == 2:
        return [[1 if fecha.isoweekday() == i else 0 for i in range(1,8)]]
    elif cod_variante == 3:
        return [[np.sin(2 * np.pi * fecha.isoweekday() / 7), np.cos(2 * np.pi * fecha.isoweekday() / 7)]]

def mostrar_widgets(clientes, clasificadores):
    productos = {0:'KS_302',1:'KS_304',2:'KS_307',3:'KS_308',4:'KS_309',5:'KS_310',6:'KS_311',7:'KS_312',8:'KS_313',9:'KS_318',10:'KS_319',11:'KS_320',12:'KS_321',13:'KS_324',14:'KS_325',15:'KS_326',16:'KS_328',17:'KS_329',18:'KS_330',19:'KS_331',20:'KS_332',21:'KS_333',22:'KS_335',23:'KS_336',24:'KS_337',25:'KS_338',26:'KS_339',27:'KS_340',28:'KS_341',29:'KS_342',30:'KS_343',31:'KS_344',32:'KS_345',33:'KS_346',34:'KS_348',35:'KS_349',36:'KS_350',37:'KS_352',38:'KS_355',39:'KS_358',40:'KS_359',41:'KS_360',42:'KS_361',43:'KS_362',44:'KS_364',45:'KS_365',46:'KS_366',47:'KS_367',48:'KS_368',49:'KS_369',50:'KS_370',51:'KS_371',52:'KS_372',53:'KS_373',54:'KS_374',55:'KS_378',56:'KS_379',57:'KS_381',58:'KS_382',59:'KS_383',60:'KS_386',61:'KS_387',62:'KS_388',63:'KS_389',64:'KS_394',65:'KS_396',66:'KS_398',67:'KS_399',68:'KS_402',69:'KS_404',70:'KS_406',71:'KS_407',72:'KS_408',73:'KS_409',74:'KS_410',75:'KS_414',76:'KS_415',77:'KS_422',78:'KS_423',79:'KS_442',80:'KS_443',81:'KS_454',82:'KS_456',83:'KS_458',84:'KS_460',85:'KS_461',86:'KS_462',87:'KS_464',88:'KS_465',89:'KS_467',90:'KS_468',91:'KS_469',92:'KS_478',93:'KS_479',94:'KS_484',95:'KS_485',96:'KS_486',97:'KS_487',98:'KS_488',99:'KS_489',100:'KS_490',101:'KS_491',102:'KS_494',103:'KS_495',104:'KS_496',105:'KS_497',106:'KS_511',107:'KS_516',108:'KS_517',109:'KS_518',110:'KS_526',111:'KS_527',112:'KS_532',113:'KS_533',114:'KS_536',115:'KS_537',116:'KS_538',117:'KS_539',118:'KS_540',119:'KS_542',120:'KS_543',121:'KS_544',122:'KS_545',123:'KS_546',124:'KS_547',125:'KS_548',126:'KS_549',127:'KS_551',128:'KS_552',129:'KS_555',130:'KS_556',131:'KS_558',132:'KS_559',133:'KS_560',134:'KS_561',135:'KS_562',136:'KS_564',137:'KS_566',138:'KS_568',139:'KS_570',140:'KS_571',141:'KS_572',142:'KS_574',143:'KS_575',144:'KS_576',145:'KS_578',146:'KS_579',147:'KS_582',148:'KS_583',149:'KS_584',150:'KS_585',151:'KS_586',152:'KS_587',153:'KS_590',154:'KS_591',155:'KS_592',156:'KS_594',157:'KS_595',158:'KS_596',159:'KS_598',160:'KS_600',161:'KS_602',162:'KS_603',163:'KS_604',164:'KS_605',165:'KS_606',166:'KS_607',167:'KS_610',168:'KS_612',169:'KS_613',170:'KS_614',171:'KS_615',172:'KS_624',173:'KS_626',174:'KS_635',175:'KS_636',176:'KS_638',177:'KS_644',178:'KS_645',179:'KS_649',180:'KS_651',181:'KS_652',182:'KS_653',183:'KS_654',184:'KS_655',185:'KS_656',186:'KS_657',187:'KS_661',188:'KS_662',189:'KS_664',190:'KS_665',191:'KS_666',192:'KS_668',193:'KS_669',194:'KS_676',195:'KS_680',196:'KS_681',197:'KS_682',198:'KS_688',199:'KS_689',200:'KS_690',201:'KS_692',202:'KS_693',203:'KS_696',204:'KS_702',205:'KS_704',206:'KS_711',207:'KS_712',208:'KS_713',209:'KS_718',210:'KS_719',211:'KS_720',212:'KS_721',213:'KS_938'}
    html_titulo = wd.HTML('<h3>Sugerencia de producto a vender</h3>')

    dpw_clientes = wd.Dropdown(options = clientes, description = 'Cliente')
    dpk_fecha_venta = wd.DatePicker(description = 'Fecha')

    btn_predecir_producto = wd.Button(description = 'Sugerir producto', button_style = 'success', icon = 'check')
    def on_btn_predecir_producto_clicked(b):
        cod_variante, clasificador = clasificadores[dpw_clientes.value]
        if cod_variante is not None:
            data = [[dpk_fecha_venta.value]]
            X = convertir_a_variante(cod_variante, data)
            prediccion = productos[clasificador.predict(X)[0]]
        else:
            prediccion = productos[clasificador]
        with out_prediccion:
            clear_output()
            print('Producto {}'.format(prediccion))

    btn_predecir_producto.on_click(on_btn_predecir_producto_clicked)

    out_prediccion = wd.Output(layout = wd.Layout(margin = '5px 0 5px 0'))

    display(wd.VBox([html_titulo, dpw_clientes, dpk_fecha_venta, btn_predecir_producto, out_prediccion]))

def main():
    clasificadores = obtener_clasificadores()
    clientes = clasificadores.keys()
    mostrar_widgets(clientes, clasificadores)

main()